In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import numpy as np
import pandas as pd
import random
from sklearn.preprocessing import LabelEncoder

In [2]:
#load testing dataset
test_data = np.load("public_set_5.npy",allow_pickle = True)

test_alert_key = test_data[:,0]

custinfo_arr = test_data[:,1]
test_custinfo = np.array([np.array(i) for i in custinfo_arr]) #convert all into array

dp_5_arr = test_data[:,2]
test_dp_5 = np.array([np.array(i) for i in dp_5_arr]) #convert all into array

ccba_5_arr = test_data[:,3]
test_ccba_5 = np.array([np.array(i) for i in ccba_5_arr]) #convert all into array

cdtx_5_arr = test_data[:,4]
test_cdtx_5 = np.array([np.array(i) for i in cdtx_5_arr]) #convert all into array

remit_5_arr = test_data[:,5]
test_remit_5 = np.array([np.array(i) for i in remit_5_arr]) #convert all into array


sarflag_arr = pd.read_csv("test_y_answer.csv").values
#capture the alertkey which is money laundry record
sarflag1_alertkey = np.array([row[0] for row in sarflag_arr if row[1]==1])

laundry_label = np.array([0,1])
normal_label = np.array([1,0])

test_sarflag = np.array([laundry_label if row[1] == 1 else normal_label for row in sarflag_arr])
print(test_sarflag)

[[1 0]
 [1 0]
 [1 0]
 ...
 [1 0]
 [1 0]
 [1 0]]


In [3]:
#find test_alertkey which is money laundry record
test_laundry_alertkey = [key for key in test_alert_key if key in sarflag1_alertkey]
print("test money laundry alert key:",test_laundry_alertkey)
print("len of test laundry records:",len(test_laundry_alertkey))

test money laundry alert key: [354939, 355091, 355152, 355724, 359668, 356602, 363320, 358453, 363896, 361617, 363033]
len of test laundry records: 11


In [5]:
#calculate class weight:
neg = 0
pos = 0
total = len(train_sarflag)

for label in train_sarflag:
    if label[0] == 1:
        neg += 1
    else:
        pos += 1

weight_for_0 = (1 / neg) * (total / 2.0)
weight_for_1 = (1 / pos) * (total / 2.0)/(3/2)

class_weight = {0: weight_for_0, 1: weight_for_1}

print(class_weight)

NameError: name 'train_sarflag' is not defined

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import math
import tensorflow as tf
import logging
tf.get_logger().setLevel(logging.ERROR)
from tensorflow.keras import models
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dropout
from tensorflow.keras import Input
#https://keras.io/api/layers/core_layers/input/
from tensorflow.keras.layers import concatenate

In [5]:
METRICS = [
      tf.keras.metrics.Recall(name='recall'),
]


In [6]:
def capture_sarflags_cost(real_laundry_key,y_pred,shuffle_alert_key):
    
    #concat alertkey with predicted probability
    alert_key_with_prob = np.hstack((np.array(shuffle_alert_key).reshape(-1,1),
                                     y_pred.reshape(-1,1)))
    
    #sorted by probability
    alert_key_with_prob = np.array(sorted(alert_key_with_prob, key=lambda x: x[1],reverse=True))
    #capture only index
    sorted_alert_key = alert_key_with_prob[:,0].tolist()
    
    distance = [0,0]
    for index in real_laundry_key:
        if distance[1] < sorted_alert_key.index(index):
            distance[0] = distance[1]
            distance[1] = sorted_alert_key.index(index)
        else:
            distance[0] = max(distance[0],sorted_alert_key.index(index))
    
    precision = (len(real_laundry_key)-1)/distance[0]
    return precision

In [7]:
def model_phase(epoch_num,comb,count):
    model = models.load_model('model_saved/my_final_model')

    y_pred = model.predict([test_custinfo,test_dp_5,test_ccba_5,test_cdtx_5,test_remit_5])
    #model.save(f'saved_model/my_model{count}')
    #tf.keras.backend.clear_session()
    return y_pred

In [10]:
#窮舉所有可能的排列組合
from itertools import combinations

def order_and_group(length):
    #because we need to keep a
    _len = length-1
    numerical_list = [i for i in range(1,_len+1,1)]
    
    possible_group = []
    for num in numerical_list:
        possible_group += list(combinations(numerical_list,num))
    
    comb_list = [[0]]
    for group in possible_group:
        temp_list = [0] + [i for i in group]
        comb_list.append(temp_list)
    return comb_list
comb_list = order_and_group(5)
print(comb_list)

[[0], [0, 1], [0, 2], [0, 3], [0, 4], [0, 1, 2], [0, 1, 3], [0, 1, 4], [0, 2, 3], [0, 2, 4], [0, 3, 4], [0, 1, 2, 3], [0, 1, 2, 4], [0, 1, 3, 4], [0, 2, 3, 4], [0, 1, 2, 3, 4]]


In [10]:
#right now num of epoch
comb_list = [0,1,4]
epoch_ = 16
pred_list= []
for _ in range(30):
    #store the result of each epoch
    result_list = []
    column_name = ["epoch_num","combination","average_precision","min_value","max_value"]
    precision_list = []
    pred = model_phase(epoch_,comb_list,_)
    precision = capture_sarflags_cost(test_laundry_alertkey,pred[:,1],test_alert_key)
    precision_list.append(precision)
    average_precision = round(sum(precision_list)/len(precision_list),5)
    max_precision = round(max(precision_list),5)
    min_precision = round(min(precision_list),5)
    pred_list.append(pred)
    print(f"epoch: {epoch_}  combination: {comb_list}  ave_precision: {average_precision}",
         f"  min_precision: {min_precision}  min_precision: {max_precision}")

epoch: 16  combination: [0, 1, 4]  ave_precision: 0.03861   min_precision: 0.03861  min_precision: 0.03861
epoch: 16  combination: [0, 1, 4]  ave_precision: 0.03861   min_precision: 0.03861  min_precision: 0.03861
epoch: 16  combination: [0, 1, 4]  ave_precision: 0.03861   min_precision: 0.03861  min_precision: 0.03861
epoch: 16  combination: [0, 1, 4]  ave_precision: 0.03861   min_precision: 0.03861  min_precision: 0.03861


KeyboardInterrupt: 

In [14]:
#load testing dataset
test_data = np.load("public_set_8.npy",allow_pickle = True)

test_alert_key = test_data[:,0]

custinfo_arr = test_data[:,1]
test_custinfo = np.array([np.array(i) for i in custinfo_arr]) #convert all into array

dp_5_arr = test_data[:,2]
test_dp_5 = np.array([np.array(i) for i in dp_5_arr]) #convert all into array

ccba_5_arr = test_data[:,3]
test_ccba_5 = np.array([np.array(i) for i in ccba_5_arr]) #convert all into array

cdtx_5_arr = test_data[:,4]
test_cdtx_5 = np.array([np.array(i) for i in cdtx_5_arr]) #convert all into array
print(test_cdtx_5)
cdtx_5_arr =[[[0 for i in range(184)] for j in range(5)] for k in range(2005)]
test_cdtx_5 = np.array([np.array(i) for i in cdtx_5_arr])

print(test_cdtx_5.shape)
remit_5_arr = test_data[:,5]
test_remit_5 = np.array([np.array(i) for i in remit_5_arr]) #convert all into array


sarflag_arr = pd.read_csv("test_y_answer.csv").values
#capture the alertkey which is money laundry record
sarflag1_alertkey = np.array([row[0] for row in sarflag_arr if row[1]==1])

laundry_label = np.array([0,1])
normal_label = np.array([1,0])

test_sarflag = np.array([laundry_label if row[1] == 1 else normal_label for row in sarflag_arr])
print(test_sarflag)

[[[0.         0.         0.         0.         0.        ]
  [0.         0.         0.         0.         0.        ]
  [0.         0.         0.         0.         0.        ]
  [0.         0.         0.         0.         0.        ]
  [0.         0.         0.         0.         0.        ]]

 [[0.         0.         0.         0.         0.        ]
  [0.         0.         0.         0.         0.        ]
  [0.         0.         0.         0.         0.        ]
  [0.         0.         0.         0.         0.        ]
  [0.         0.         0.         0.         0.        ]]

 [[0.         0.         0.         0.         0.        ]
  [0.         0.         0.         0.         0.        ]
  [0.         0.         0.         0.         0.        ]
  [0.         0.         0.         0.         0.        ]
  [0.         0.         0.         0.         0.        ]]

 ...

 [[1.         0.         0.         1.         0.32076797]
  [0.         1.         0.         1.      

In [15]:
model = models.load_model('model_saved/my_final_model')
y_pred = model.predict([test_custinfo,test_dp_5,test_ccba_5,test_cdtx_5,test_remit_5])
print(y_pred)

[[9.1596800e-01 8.4032007e-02]
 [1.0000000e+00 6.5761107e-12]
 [1.0000000e+00 4.9072335e-10]
 ...
 [1.7507574e-02 9.8249245e-01]
 [4.8876104e-01 5.1123893e-01]
 [9.4295776e-01 5.7042185e-02]]


In [19]:
print(len(pred_list))
pred = y_pred

4


In [20]:
ans_csv = pd.DataFrame([])
ans_csv["alert_key"] = test_data[:,0]
display(ans_csv)
print(ans_csv.shape)
ans_csv["probability"] = pred[:,1]
display(ans_csv)
ans_csv.to_csv("ans_csv1204.csv",index=False)

handout_data = pd.read_csv("D:\Python_projects\DataMining\yushan\預測的案件名單及提交檔案範例.csv")
handout_data = handout_data.merge(ans_csv,on="alert_key",how="left")
display(handout_data)
handout_data =  handout_data.drop(columns="probability_x")
handout_data = handout_data.rename(columns={"probability_y":"probability"})
handout_data = handout_data.fillna(0)
display(handout_data)
handout_data.to_csv("handout_dataVer8private.csv",index=False)


,alert_key
0,370282
1,369792
2,370155
3,369814
4,370230
...,...
2000,378249
2001,378060
2002,378409
2003,377954


(2005, 1)


,alert_key,probability
0,370282,8.403201e-02
1,369792,6.576111e-12
2,370155,4.907234e-10
3,369814,7.594651e-02
4,370230,2.931321e-02
...,...,...
2000,378249,1.636838e-01
2001,378060,7.116236e-07
2002,378409,9.824924e-01
2003,377954,5.112389e-01


,alert_key,probability_x,probability_y
0,357307,0.000017,NaN
1,376329,0.000324,6.687203e-07
2,373644,0.000372,1.910048e-01
3,357668,0.000489,NaN
4,354443,0.000526,NaN
...,...,...,...
3845,364485,0.997702,NaN
3846,363155,0.998987,NaN
3847,368710,0.999694,7.126819e-01
3848,358067,0.999821,NaN


,alert_key,probability
0,357307,0.000000e+00
1,376329,6.687203e-07
2,373644,1.910048e-01
3,357668,0.000000e+00
4,354443,0.000000e+00
...,...,...
3845,364485,0.000000e+00
3846,363155,0.000000e+00
3847,368710,7.126819e-01
3848,358067,0.000000e+00


In [14]:
from sklearn.metrics import classification_report

y_pred = [0 if label[0]> label[1] else 1 for label in pred]
y_true = [0 if label[0]> label[1] else 1 for label in test_sarflag]

print(classification_report(y_true,y_pred))
print(len(pred))

y_pred_index = [i for i in range(len(y_pred)) if y_pred[i]==1]
y_true_index = [i for i in range(len(y_true)) if y_true[i]==1]
print("predict index:",y_pred_index)
print("true index:",y_true_index)

              precision    recall  f1-score   support

           0       1.00      0.83      0.91      1834
           1       0.03      0.73      0.05        11

    accuracy                           0.83      1845
   macro avg       0.51      0.78      0.48      1845
weighted avg       0.99      0.83      0.90      1845

1845
predict index: [4, 6, 11, 19, 24, 29, 43, 47, 54, 62, 63, 77, 81, 82, 96, 97, 114, 133, 135, 138, 146, 152, 155, 162, 165, 167, 177, 181, 183, 189, 192, 207, 212, 214, 218, 224, 226, 230, 231, 232, 237, 245, 247, 262, 269, 273, 275, 284, 294, 304, 310, 313, 329, 334, 335, 344, 345, 347, 351, 365, 367, 368, 370, 375, 377, 381, 388, 396, 397, 400, 412, 414, 415, 422, 424, 438, 443, 460, 461, 462, 463, 466, 467, 469, 474, 477, 478, 484, 489, 499, 500, 502, 506, 513, 518, 525, 526, 534, 538, 540, 542, 548, 550, 555, 558, 560, 570, 579, 580, 590, 592, 595, 604, 619, 636, 659, 662, 664, 668, 671, 684, 692, 693, 703, 707, 710, 719, 720, 731, 742, 743, 755, 759, 766, 